# LangGraph Agentic Rag

## Imports

In [4]:
import json
import operator
import os
import re
from typing import Dict, List, Optional, Union

from dotenv import load_dotenv
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.runnables import RunnableLambda
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

load_dotenv()


print(os.getenv("MODEL_NAME"))
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("OPENAI_API_BASE"))

mistral-large-latest
NuNTGZkdk67xWT4RcEYzCYtAHvsPpK71
https://api.mistral.ai/v1


## Define Models

In [5]:
response_model = ChatOpenAI(
    model=os.getenv("MODEL_NAME"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    max_tokens=8000,
    temperature=0,
)


grader_model = ChatOpenAI(
    model=os.getenv("MODEL_NAME"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    max_tokens=8000,
    temperature=0,
)

# 0. Process des documents

In [8]:
PDF_PATH = "./data"
EMBEDDING_MODEL_PATH = "./models/sentence_transformers"
LLM_MODEL_PATH = "./models/Falcon3-3B-Instruct"

In [9]:
# Load les pdfs
def load_pdf_from_diretory(dir_path: str):
    """Load local pdf file as Langchain document object"""
    documents = []
    for filename in os.listdir(dir_path):
        loader = PyPDFLoader(os.path.join(dir_path, filename))
        documents.extend(loader.load())
    return documents


docs_before_split = load_pdf_from_diretory(PDF_PATH)
print(docs_before_split[0].page_content)

Code civil
Titre préliminaire : De la publication, des effets et de l'application des
lois en général
Article 1
 
Les lois et, lorsqu'ils sont publiés au Journal officiel de la République française, les actes administratifs
entrent en vigueur à la date qu'ils fixent ou, à défaut, le lendemain de leur publication. Toutefois, l'entrée en
vigueur de celles de leurs dispositions dont l'exécution nécessite des mesures d'application est reportée à la
date d'entrée en vigueur de ces mesures.
 
 
En cas d'urgence, entrent en vigueur dès leur publication les lois dont le décret de promulgation le prescrit et
les actes administratifs pour lesquels le Gouvernement l'ordonne par une disposition spéciale.
 
 
Les dispositions du présent article ne sont pas applicables aux actes individuels.
 
Article 2
 
La loi ne dispose que pour l'avenir ; elle n'a point d'effet rétroactif.
 
 
Article 3
 
Les lois de police et de sûreté obligent tous ceux qui habitent le territoire.
 
 
Les immeubles, même ceux 

In [ ]:
huggingface_embedding_model = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDING_MODEL_PATH,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# Initialiser le chunker
text_splitter = SemanticChunker(huggingface_embedding_model)

# Decoupe des data en chunks
docs_after_split = text_splitter.split_documents(docs_before_split)

vectorstore = FAISS.from_documents(docs_after_split, huggingface_embedding_model)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Créer l'outil de récupération
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retrieve_blog_posts",
    description="Search and return information about the python documentation.",
)

# Lier l'outil au modèle
model_with_tools = response_model.bind_tools([retriever_tool])


result = retriever_tool.invoke({"query": "python command to create a virtual env"})
print(result)

In [ ]:
# Définition des types et fonctions de base
class MessagesState(Dict):
    messages: List[Union[HumanMessage, AIMessage, ToolMessage]]

def safe_get_messages(state: MessagesState) -> List[Union[HumanMessage, AIMessage, ToolMessage]]:
    """Récupère les messages de manière sûre, avec des valeurs par défaut si nécessaire"""
    return state.get("messages", [])

In [ ]:
def grade_documents(state: MessagesState) -> str:
    """Évalue la pertinence des documents récupérés de manière plus robuste"""
    messages = safe_get_messages(state)

    # Vérifier qu'il y a assez de messages
    if len(messages) < 2:
        print("Warning: Not enough messages to evaluate relevance")
        return "rewrite_question"

    try:
        question = messages[0].content
        context = messages[-1].content

        # Vérifier que le contenu existe
        if not question or not context:
            return "rewrite_question"

        # Créer le prompt pour l'évaluation
        prompt = f"""
        Évaluez si ce document est pertinent pour répondre à la question sur les environnements virtuels Python.

        Question: {question}
        Document: {context}

        Répondez uniquement par 'yes' si le document est pertinent, ou 'no' s'il ne l'est pas.
        """

        # Utiliser le modèle pour évaluer
        evaluation = grader_model.invoke([HumanMessage(content=prompt)])

        # Post-traitement pour s'assurer de la pertinence
        if evaluation and hasattr(evaluation, 'content') and "yes" in evaluation.content.lower():
            return "generate_answer"
        else:
            return "rewrite_question"

    except Exception as e:
        print(f"Error in grade_documents: {str(e)}")
        return "rewrite_question"

In [ ]:






def generate_query_or_response(state: MessagesState) -> MessagesState:
    """Version plus robuste de la génération de réponse"""
    messages = safe_get_messages(state)

    if not messages:
        return {"messages": [AIMessage(content="Je n'ai pas reçu de question valide.")]}

    try:
        # Convertir les messages en format attendu par le modèle
        input_messages = []
        for msg in messages:
            if isinstance(msg, HumanMessage):
                input_messages.append({"role": "user", "content": msg.content})
            elif isinstance(msg, AIMessage):
                tool_calls = []
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    tool_calls = msg.tool_calls
                input_messages.append({
                    "role": "assistant",
                    "content": msg.content,
                    "tool_calls": tool_calls
                })
            elif isinstance(msg, ToolMessage):
                input_messages.append({
                    "role": "tool",
                    "content": msg.content,
                    "tool_call_id": msg.tool_call_id
                })

        # Appeler le modèle avec les outils
        response = response_model.bind_tools([retriever_tool]).invoke(input_messages)

        # Traiter la réponse
        if isinstance(response, dict):
            content = response.get("content", "")
            tool_calls = response.get("tool_calls", [])
            new_message = AIMessage(content=content)
            if tool_calls:
                new_message.tool_calls = tool_calls
            new_messages = messages + [new_message]
        else:
            new_messages = messages + [AIMessage(content=str(response))]

        return {"messages": new_messages}

    except Exception as e:
        print(f"Error in generate_query_or_response: {str(e)}")
        return {"messages": messages + [AIMessage(content="Une erreur est survenue lors du traitement de votre demande.")]}

# Configuration du graphe avec gestion d'erreurs
try:
    workflow = StateGraph(MessagesState)

    # Ajouter les nœuds avec vérification
    workflow.add_node("generate_query_or_response", generate_query_or_response)
    workflow.add_node("retrieve", ToolNode([retriever_tool]))
    workflow.add_node("rewrite_question", rewrite_question)
    workflow.add_node("generate_answer", generate_answer)

    # Définir les arêtes avec vérification
    workflow.add_edge(START, "generate_query_or_response")

    def should_use_tools(state: MessagesState) -> str:
        """Fonction conditionnelle plus robuste"""
        messages = safe_get_messages(state)
        if not messages:
            return END

        last_message = messages[-1]
        if isinstance(last_message, AIMessage) and hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "retrieve"
        return END

    workflow.add_conditional_edges(
        "generate_query_or_response",
        should_use_tools,
        {
            "retrieve": "retrieve",
            END: END
        }
    )

    workflow.add_conditional_edges(
        "retrieve",
        grade_documents,
        {
            "generate_answer": "generate_answer",
            "rewrite_question": "rewrite_question"
        }
    )

    workflow.add_edge("generate_answer", END)
    workflow.add_edge("rewrite_question", "generate_query_or_response")

    # Compiler le graphe
    graph = workflow.compile()

    # Exécuter avec une entrée valide
    initial_input = {
        "messages": [
            HumanMessage(content="What is a virtual environment in Python?")
        ]
    }

    result = graph.invoke(initial_input)

    # Afficher le résultat de manière sûre
    if result and isinstance(result, dict) and "messages" in result:
        final_messages = result["messages"]
        if final_messages:
            final_message = final_messages[-1]
            if isinstance(final_message, (AIMessage, HumanMessage, ToolMessage)):
                print("Final response:", final_message.content)
            else:
                print("Final response:", str(final_message))
        else:
            print("No messages in the final result")
    else:
        print("Invalid final result format")

except Exception as e:
    print(f"Error in workflow setup: {str(e)}")
    # Afficher des informations de débogage supplémentaires
    print("Debug info:")
    print(f"Type of error: {type(e)}")
    if hasattr(e, 'message'):
        print(f"Error message: {e.message}")
